In [ ]:
from qiskit import QuantumRegister,ClassicalRegister,QuantumCircuit, execute
from qiskit import Aer, IBMQ
from qiskit import tools
from qiskit.test import mock 
import qiskit
from qiskit.quantum_info.operators import Operator
from qiskit.compiler import transpile
from qiskit.visualization import plot_gate_map
from qiskit.providers.aer import noise
import scipy.linalg as linalg
import numpy as np
from scipy.linalg import expm
from numpy.linalg import svd
import csv as csv
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import json
import scipy.sparse as sp
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy.linalg import expm
from numpy import pi as pi
import matplotlib.pyplot as plt
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
import datetime 
import argparse
import random
import pickle
from importlib import reload
import sys
import gvar as gv
import lsqfit
import pandas as pd
import json
from qiskit.quantum_info.operators.channel import Choi, Kraus
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')

In [ ]:

#-------------------------------------------------------
# Like job_monitor but all contained in a single overlay
from qiskit.tools.jupyter import job_watcher
%qiskit_job_watcher
#-------------------------------------------------------
IBMQ.providers()

In [ ]:
c, s = np.cos(np.pi / 4), -1.0j * np.sin(np.pi / 4)
x = np.array([[0, 1], [1, 0]])
y = np.array([[0, -1j], [1j, 0]])
z = np.array([[1, 0], [0, -1]])
xi = 0.05
op1 = expm(-1.0j * np.pi / 4 * (np.kron(x, z) + xi * np.kron(z, np.identity(2))))
op2 = (c * np.identity(4) - s * np.kron(np.identity(2), z))
op3 = (c * np.identity(4) - s * np.kron(x, np.identity(2)))
cx = np.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])
op1 @ op2 @ op3 @ cx

# ampdamp = noise.
backend = Aer.get_backend('aer_simulator_density_matrix')
c, s = np.cos(np.pi / 4), -1.0j * np.sin(np.pi / 4)
x_op = np.array([[0, 1], [1, 0]])
z_op = np.array([[1, 0], [0, -1]])
xi = 0.01
op1 = expm(-1.0j * np.pi / 4 * (np.kron(x_op, z_op) + xi * np.kron(z_op, np.identity(2))))
op2 = (c * np.identity(4) - s * np.kron(np.identity(2), z_op))
op3 = (c * np.identity(4) - s * np.kron(x_op, np.identity(2)))
cx = np.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])
op1 @ op2 @ op3 @ cx
noise_model = noise.NoiseModel()
# coherent = noise.coherent_unitary_error(op1 @ op2 @ op3 @ cx)

noise_model = noise.NoiseModel()

print(np.round(op1 @ op2 @ op3, decimals=3))
print(cx)
noise_model.add_all_qubit_quantum_error(noise.coherent_unitary_error(op1 @ op2 @ op3 @ cx), 'cx')
# cx @ op2.conjugate().transpose() @ op3.conjugate().transpose() / (0.5 - 0.5j) * 1 / np.sqrt(2)

In [ ]:
TwirlCNOT = {(0, 0): (0, 0), (0, 1): (0, 1), (0, 2): (3, 2), (0, 3): (3, 3),
             (1, 0): (1, 1), (1, 1): (1, 0), (1, 2): (2, 3), (1, 3): (2, 2),
             (2, 0): (2, 1), (2, 1): (2, 0), (2, 2): (1, 3), (2, 3): (1, 2),
             (3, 0): (3, 0), (3, 1): (3, 1), (3, 2): (0, 2), (3, 3): (0, 3)}


def Twirl(qc):
    rand_set = np.random.randint(0, 4, size=7)
    gates = [lambda x: 0, qc.x, qc.y, qc.z]
    for i in range(7):
        gates[rand_set[i]](i)
    return (qc, rand_set)


def Comp(qc, cxpairs, specs):
    gates = [lambda x: 0, qc.x, qc.y, qc.z]
    for pair in cxpairs.keys():
        c, t = pair
        comp = TwirlCNOT[cxpairs[pair]]
        gates[comp[0]](c)
        gates[comp[1]](t)
    for spec in specs:
        qubit, gate = spec
        gates[gate](qubit)
    return qc
    
def Trotter(qc, epsilon, mass, time_steps, NCX):
    # operator layout
    #  0 - 1 - 2   m1 - m2 - g12
    #      |            |
    #      3            g23
    #      |            |
    #  4 - 5 - 6   m4 - m3 - g34
    
    for t in range(time_steps):
        #============#
        # mass terms #
        #============#
        for i in range(4):
            qc.rz((-1) ** i * mass * epsilon / 2, 2 * i)
        #=============#
        # gauge terms #
        #=============#
        for i in range(3):
            qc.rx(epsilon, 2 * i + 1)
        #================#
        # fermion term 1 #
        #================#
        qc.z([2, 4])
        qc.sxdg([1, 5])
        qc.s([1, 5])
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(2, 1)
            qc.cx(4, 5)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(2, 1): (twirl_set[2], twirl_set[1]),
                      (4, 5): (twirl_set[4], twirl_set[5])}
            spectators = [(a, twirl_set[a]) for a in [0, 3, 6]]
            qc = Comp(qc, cxpairs, spectators)
        #================================
        qc.sx([2, 4])
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(2, 0)
            qc.cx(4, 6)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(2, 0): (twirl_set[2], twirl_set[0]),
                      (4, 6): (twirl_set[4], twirl_set[6])}
            spectators = [(a, twirl_set[a]) for a in [1, 3, 5]]
            qc = Comp(qc, cxpairs, spectators)
        #===============================
        qc.rx(epsilon / 2, [2, 4])
        qc.ry(epsilon / 2, [0, 6])
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(2, 0)
            qc.cx(4, 6)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(2, 0): (twirl_set[2], twirl_set[0]),
                      (4, 6): (twirl_set[4], twirl_set[6])}
            spectators = [(a, twirl_set[a]) for a in [1, 3, 5]]
            qc = Comp(qc, cxpairs, spectators)
        #===============================
        qc.sxdg([2, 4])
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(2, 1)
            qc.cx(4, 5)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(2, 1): (twirl_set[2], twirl_set[1]),
                      (4, 5): (twirl_set[4], twirl_set[5])}
            spectators = [(a, twirl_set[a]) for a in [0, 3, 6]]
            qc = Comp(qc, cxpairs, spectators)
        #================================
        qc.sdg([1, 5])
        qc.sx([1, 5])
        qc.z([2, 4])
        #===========#
        # Fhop pt2  #
        #===========#
        qc.h(3)
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(3, 2)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(3, 2): (twirl_set[3], twirl_set[2])}
            spectators = [(a, twirl_set[a]) for a in [0, 1, 4, 5, 6]]
            qc = Comp(qc, cxpairs, spectators)
        #================================
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(3, 4)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(3, 4): (twirl_set[3], twirl_set[4])}
            spectators = [(a, twirl_set[a]) for a in [0, 1, 2, 5, 6]]
            qc = Comp(qc, cxpairs, spectators)
        #================================
        qc.rx(-epsilon / 2, 3)
        qc.h(2)
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(3, 2)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(3, 2): (twirl_set[3], twirl_set[2])}
            spectators = [(a, twirl_set[a]) for a in [0, 1, 4, 5, 6]]
            qc = Comp(qc, cxpairs, spectators)
        #================================
        qc.h(2)
        qc.s(2)
        qc.z(3)
        qc.h(4)
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(3, 4)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(3, 4): (twirl_set[3], twirl_set[4])}
            spectators = [(a, twirl_set[a]) for a in [0, 1, 2, 5, 6]]
            qc = Comp(qc, cxpairs, spectators)
        #================================
        qc.h(4)
        qc.s(4)
        qc.rx(-epsilon / 2, 3)
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(3, 4)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(3, 4): (twirl_set[3], twirl_set[4])}
            spectators = [(a, twirl_set[a]) for a in [0, 1, 2, 5, 6]]
            qc = Comp(qc, cxpairs, spectators)
        #================================
        #=========================
        for k in range(NCX):
            #TWIRLING
            qc, twirl_set = Twirl(qc)
            #=========================
            qc.barrier()
            qc.cx(3, 2)
            qc.barrier()
            #===========================
            # UNTWIRLING
            cxpairs = {(3, 2): (twirl_set[3], twirl_set[2])}
            spectators = [(a, twirl_set[a]) for a in [0, 1, 4, 5, 6]]
            qc = Comp(qc, cxpairs, spectators)
        #================================
        qc.h(3)
        qc.sdg([2, 4])
        #============#
        # mass terms #
        #============#
        for i in range(4):
            qc.rz((-1) ** i * mass * epsilon / 2, 2 * i)
        #=============#
        # gauge terms #
        #=============#
        for i in range(3):
            qc.rx(epsilon, 2 * i + 1)
#         qc.save_density_matrix(label=str((t,)))
    return qc

In [ ]:
circuits = []
mass = 1
eps = 0.6
ncircuits = 15
nt = 5
NCX = [1,3,5]

for k in NCX:
    for i in range(ncircuits):
        for t in range(nt):
            qcr = QuantumCircuit(7,7)
            qcr.h([1, 3, 5])
            qcr.z([1, 3, 5])
        # #     qcr.h([1, 3, 5])
            qcr.x([2, 4])
            qcr.z(3)
            qcrt = Trotter(qcr, eps, mass, t + 1,k)
            qcrt.measure([i for i in range(7)], [i for i in range(7)])
            qcrt = (transpile(qcrt, initial_layout=[0, 2, 1, 3, 5, 4, 6], optimization_level=2,
                              backend=provider.get_backend('ibm_perth')))
            circuits.append(qcrt)

            qcr = QuantumCircuit(7,7)
    #         qcr.h([1, 3, 5])
    #         qcr.z([1, 3, 5])
    #     # #     qcr.h([1, 3, 5])
    #         qcr.x([2, 4])
    #         qcr.z(3)
            qcrt = Trotter(qcr, 0, mass, t + 1,k)
            qcrt.measure([i for i in range(7)], [i for i in range(7)])
            qcrt.draw()
            qcrt = (transpile(qcrt, initial_layout=[0, 2, 1, 3, 5, 4, 6], optimization_level=2,
                              backend=provider.get_backend('ibm_perth')))
            circuits.append(qcrt)

In [ ]:
nshots=1024
job1 = execute(circuits[:75], backend=provider.get_backend('ibm_perth'), shots=nshots)
job2 = execute(circuits[75:150], backend=provider.get_backend('ibm_perth'), shots=nshots)
job3 = execute(circuits[150:225], backend=provider.get_backend('ibm_perth'), shots=nshots)
job4 = execute(circuits[225:300], backend=provider.get_backend('ibm_perth'), shots=nshots)
job5 = execute(circuits[300:375], backend=provider.get_backend('ibm_perth'), shots=nshots)
job6 = execute(circuits[375:], backend=provider.get_backend('ibm_perth'), shots=nshots)

# job = execute(circuits, backend=Aer.get_backend('aer_simulator_density_matrix'), shots=nshots, 
#                initial_layout=[0, 2, 1, 3, 5, 4, 6])

In [ ]:
counts = []
for i in range(75):
    counts.append(job1.result().get_counts(i))
    
for i in range(75):
    counts.append(job2.result().get_counts(i))
    
for i in range(75):
    counts.append(job3.result().get_counts(i))
    
for i in range(75):
    counts.append(job4.result().get_counts(i))
    
for i in range(75):
    counts.append(job5.result().get_counts(i))
    
for i in range(75):
    counts.append(job6.result().get_counts(i))

In [ ]:
ops = [np.kron(np.identity(2 ** i), np.kron(np.diag([1, -1]),
                                          np.identity(2 ** (6 - i)))) for i in range(7)]
ncircuits=15
nt=5
NCX = [1,3,5]
nshots = 1024
AllRich = []
AllScale = []

index = 0
    

for k in range(len(NCX)):
    
    richs = np.zeros((ncircuits, nt, 7))
    scales = np.zeros((ncircuits, nt, 7))

    for j in range(ncircuits):

        for i in range(nt):

            counts_rich = counts[index]
            index += 1
            counts_scale = counts[index]
            index += 1

            for key in counts_rich.keys():
                for site in range(7):
                    if key[site] == '0':
                        richs[j, i, site] += counts_rich[key] / nshots
                    else:
                        richs[j, i, site] -= counts_rich[key] / nshots


            for key in counts_scale.keys():
                for site in range(7):
                    if key[site] == '0':
                        scales[j, i, site] += counts_scale[key] / nshots
                    else:
                        scales[j, i, site] -= counts_scale[key] / nshots


    AllRich.append(richs)
    AllScale.append(scales)


In [ ]:
AllRichavg = []
AllScaleavg = []
AllRichavgErr = []

for k in range(len(NCX)):
    AllRichavg.append(gv.gvar(np.mean(AllRich[k], axis=0), np.std(AllRich[k], axis=0) / np.sqrt(ncircuits)))
    AllScaleavg.append(gv.gvar(np.mean(AllScale[k], axis=0), np.std(AllScale[k], axis=0) / np.sqrt(ncircuits)))
    AllRichavgErr.append(AllRichavg[k]/AllScaleavg[k])

In [ ]:
prior = {'f(A)': gv.BufferDict.uniform('f', 1, -1), 'log(B)': gv.gvar(0, 2000)}
exponential = lambda x, p: [p['A'] * gv.exp(-p['B'] * x[i]) for i in range(len(x))]
nt = 5
eps = 0.6

expfits = []
for i in range(nt):
    fit0 = lsqfit.nonlinear_fit(data=(NCX,[AllRichavgErr[j][i][0] for j in range(len(NCX))]), 
                            prior=prior, fcn=exponential, debug=True)
    fit1 = lsqfit.nonlinear_fit(data=(NCX,[AllRichavgErr[j][i][1] for j in range(len(NCX))]), 
                            prior=prior, fcn=exponential, debug=True)
    fit2 = lsqfit.nonlinear_fit(data=(NCX,[AllRichavgErr[j][i][2] for j in range(len(NCX))]), 
                            prior=prior, fcn=exponential, debug=True)
    
    fit3 = lsqfit.nonlinear_fit(data=(NCX,[AllRichavgErr[j][i][3] for j in range(len(NCX))]), 
                            prior=prior, fcn=exponential, debug=True)
    
    fit4 = lsqfit.nonlinear_fit(data=(NCX,[AllRichavgErr[j][i][4] for j in range(len(NCX))]), 
                            prior=prior, fcn=exponential, debug=True)
    
    fit5 = lsqfit.nonlinear_fit(data=(NCX,[AllRichavgErr[j][i][5] for j in range(len(NCX))]), 
                            prior=prior, fcn=exponential, debug=True)
    
    fit6 = lsqfit.nonlinear_fit(data=(NCX,[AllRichavgErr[j][i][6] for j in range(len(NCX))]), 
                            prior=prior, fcn=exponential, debug=True)
    expfits.append([fit0,fit1,fit2,fit3,fit4,fit5,fit6])
    

In [ ]:
nt=5
fig, ax = plt.subplots(ncols=4, nrows=2, sharex=True)
eps = 0.6
for i in range(7):
    axis = ax[i//4,i%4]
#     axis.errorbar(np.linspace(1, nt, nt) * eps, [gv.mean(AllRichavg[0][k][i]) for k in range(nt)],
#                   yerr=[gv.sdev(AllRichavg[0][k][i]) for k in range(nt)], fmt='.', markersize=10, capsize=5, 
#                   label='Twirled')
    
    axis.errorbar(np.linspace(1, nt, nt) * eps, [gv.mean(AllRichavgErr[0][k][i]) for k in range(nt)],
                  yerr=[gv.sdev(AllRichavgErr[0][k][i]) for k in range(nt)], fmt='.', markersize=10, capsize=5, 
                  label='Twirled & Rescaled')
    
    axis.errorbar(np.linspace(1, nt, nt) * eps,[gv.mean(expfits[s][i].p['A']) for s in range(nt)],
                  yerr=[gv.sdev(expfits[s][i].p['A']) for s in range(nt)], fmt='.', markersize=10, capsize=5,
                  label='Richardson Extrapolation and Rescaling (fitted data)')

#     for k in range(len(NCX)):
#         axis.errorbar(np.linspace(1, nt, nt) * eps, [gv.mean(AllScaleavg[k][t][i]) for t in range(nt)],
#                       yerr=[gv.sdev(AllScaleavg[k][t][i]) for t in range(nt)], fmt='.', capsize=3, 
#                       label='Rescaling circuit '+str(1+(2*k))+'CX')
#         axis.errorbar(np.linspace(1, nt, nt) * eps, [gv.mean(AllRichavg[k][t][i]) for t in range(nt)],
#                       yerr=[gv.sdev(AllRichavg[k][t][i]) for t in range(nt)], fmt='.', markersize=10, capsize=5,
#                       label='Richardson Circuit '+str(1+(2*k))+'CX')



    axis.plot(np.linspace(1, nt, nt) * eps, exptrots[:nt, i], label='Expected', color='k',linestyle=':')
    axis.set_ylim([-1, 1])
    axis.set_xlabel('$\epsilon$',fontsize=15)
    axis.set_ylabel('$<Z_'+str(i)+'>$',fontsize=13)
    axis.set_title('Qubit '+str(i),fontsize=20)
    axis.legend(prop={'size': 9})
fig.set_size_inches((25, 13))

In [ ]:
colors = ['#4287f5','#f5c242','#53d42c','#a66621','#732bb5','#ed4640','#d453c5']
fig3, ax = plt.subplots(ncols=3, nrows=2, sharex=True)
fig3.set_size_inches((25, 18))
for i in range(nt):
    labls = ["${\chi}^2/[dof]$ q"+str(j)+" = "+str.format('{0:.3f}', expfits[i][j].chi2/3) for j in range(7)]
    axis = ax[i//3,i%3]
    axis.set_ylim([-1.5, 1.5])
    axis.set_xlim([0, 35])
    axis.set_xlabel('# CXs',fontsize=12)
    axis.set_ylabel('$<Z>$',fontsize=12)
    axis.set_title('Step '+str(i),fontsize=15)
    
        
    for j in [0,2,4,6]:
        
        axis.plot(range(35), [exptrots[i, j] for k in range(35)], ls=':', marker='', color='grey')
        axis.annotate('$<Z_'+str(j)+'>$', xy=(15,exptrots[i, j]),fontsize=15,color=colors[j])

        axis.plot(range(35),gv.mean(exponential(range(35),expfits[i][j].p)), color='k',linewidth=0.5)
        axis.errorbar(NCX, [gv.mean(AllRichavgErr[k][i][j]) for k in range(len(NCX))],
                  yerr=[gv.sdev(AllRichavgErr[k][i][j]) for k in range(len(NCX))], fmt='.', markersize=10, 
                  capsize=5, color=colors[j], linestyle='', label='Qubit '+str(j))
        axis.fill_between(range(35), 
                          gv.mean(exponential(range(35),expfits[i][j].p))-
                          gv.sdev(exponential(range(35),expfits[i][j].p)), 
                          gv.mean(exponential(range(35),expfits[i][j].p))+
                          gv.sdev(exponential(range(35),expfits[i][j].p)),alpha=0.3,color=colors[j])
        axis.annotate(labls[j], xy=(0.5,1.4-(0.05*j)),fontsize=8)
    
        axis.legend(prop={'size': 10})